## 3. Cluster-preparation and graph generation

Preparation for clustering(we convert the itemsets of shingles/keywords/descriptors and calculate a
similarity measure between each itemsets -> this in turn forms a fully connected
graph(edges == similarity, nodes == games) which we can then prune by removing the edges where
the weight is below a threshold)

### imports

In [2]:
import polars as pl
import numpy as np
import h5py

import itertools
from pathlib import Path
from collections import defaultdict
import re
import random
import spacy



DATA = Path('./data')

GEN_DATA = DATA / "gen"
RAW_DATA = DATA / "raw"

nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"]) 

### loading data

In [3]:
from statistics import mean

df = pl.read_csv(RAW_DATA / 'games_detailed_info2025.csv')

df.head()

,type,id,thumbnail,image,alternate,description,yearpublished,minplayers,maxplayers,suggested_num_players,suggested_playerage,suggested_language_dependence,playingtime,minplaytime,maxplaytime,minage,boardgamecategory,boardgamemechanic,boardgamefamily,boardgameexpansion,boardgameaccessory,boardgamecompilation,boardgameimplementation,boardgamedesigner,boardgameartist,boardgamepublisher,usersrated,average,bayesaverage,Board Game Rank,Strategy Game Rank,Family Game Rank,stddev,median,owned,trading,wanting,wishing,numcomments,numweights,averageweight,boardgameintegration,Abstract Game Rank,Party Game Rank,Thematic Rank,War Game Rank,Customizable Rank,Children's Game Rank,RPG Item Rank,Accessory Rank,name
i64,str,i64,str,str,str,str,i64,i64,i64,str,str,str,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,i64,f64,f64,i64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,f64,str,f64,f64,f64,f64,f64,str,str,str,str
0,"""boardgame""",13,"""https://cf.geekdo-images.com/P…","""https://cf.geekdo-images.com/P…","""['Catan', 'Catan (Колонизаторы…","""In CATAN (formerly The Settler…",1995,3,4,"""[{'@numplayers': '1', 'result'…","""[{'@value': '2', '@numvotes': …","""[{'@level': '1', '@value': 'No…",120,60,120,10,"""['Economic', 'Negotiation']""","""['Chaining', 'Dice Rolling', '…","""['Animals: Sheep', 'Components…","""['20 Jahre Darmstadt Spielt', …","""['Catan x Goat Simulator 3: Re…","""[""CATAN 3D Collector's Edition…","""['Baden-Württemberg Catan', 'C…","""['Klaus Teuber']""","""['Volkan Baga', 'Tanja Donner'…","""['KOSMOS', '64 Ounce Games', '…",132477,7.09526,6.91526,573,533.0,196.0,1.49966,0,218546,2264,518,7367,22600,8299,2.2881,null,null,null,null,null,null,null,null,null,"""CATAN"""
1,"""boardgame""",822,"""https://cf.geekdo-images.com/o…","""https://cf.geekdo-images.com/o…","""['Carcassonne Jubilee Edition'…","""Carcassonne is a tile placemen…",2000,2,5,"""[{'@numplayers': '1', 'result'…","""[{'@value': '2', '@numvotes': …","""[{'@level': '6', '@value': 'No…",45,30,45,7,"""['Medieval', 'Territory Buildi…","""['Area Majority / Influence', …","""['Category: Dized Tutorial', '…","""['20 Jahre Darmstadt Spielt', …","""['The Adults of Carcassonne', …","""['Carcassonne Big Box', 'Carca…","""['The Ark of the Covenant', 'C…","""['Klaus-Jürgen Wrede']""","""['Marcel Gröber', 'Doris Matth…","""['Hans im Glück', '64 Ounce Ga…",131182,7.41145,7.29556,230,null,55.0,1.31135,0,204049,1995,656,9787,22150,8414,1.8894,"""['Carcassonne: Wheel of Fortun…",null,null,null,null,null,null,null,null,"""Carcassonne"""
2,"""boardgame""",30549,"""https://cf.geekdo-images.com/S…","""https://cf.geekdo-images.com/S…","""['EPIZOotic', 'Pandemia', 'Pan…","""In Pandemic, several virulent …",2008,2,4,"""[{'@numplayers': '1', 'result'…","""[{'@value': '2', '@numvotes': …","""[{'@level': '6', '@value': 'No…",45,45,45,8,"""['Medical']""","""['Action Points', 'Cooperative…","""['Components: Map (Global Scal…","""['Pandemic: Gen Con 2016 Promo…","""['Pandemic: Folded Space Inser…",null,"""['Fall of Rome', 'Iberia', 'Pa…","""['Matt Leacock']""","""['Josh Cappel', 'Christian Han…","""['Z-Man Games', '(Unknown)', '…",128935,7.52913,7.42156,158,168.0,32.0,1.33643,0,211600,3228,620,10981,19897,6138,2.3974,null,null,null,null,null,null,null,null,null,"""Pandemic"""
3,"""boardgame""",68448,"""https://cf.geekdo-images.com/3…","""https://cf.geekdo-images.com/3…","""['7 csoda', '7 Cudów Świata', …","""You are the leader of one of t…",2010,2,7,"""[{'@numplayers': '1', 'result'…","""[{'@value': '2', '@numvotes': …","""[{'@level': '1', '@value': 'No…",30,30,30,10,"""['Ancient', 'Card Game', 'City…","""['Closed Drafting', 'Hand Mana…","""['Ancient: Babylon', 'Ancient:…","""['7 Wonders: Armada', '7 Wonde…","""['7 Wonders: Eurohell Design C…",null,"""['7 Wonders (Second Edition)',…","""['Antoine Bauza']""","""['Dimitri Chappuis', 'Miguel C…","""['Repos Production', 'ADC Blac…",107506,7.67463,7.56393,101,111.0,18.0,1.27648,0,147129,1896,979,14247,16690,5365,2.3171,null,null,null,null,null,n

In [8]:
df = df.with_columns(pl.col("description").fill_null(""))
desc = df["description"]

mean(map(len, desc))

1174.0120590352772

In [9]:
data = list(zip(df["id"], df["description"]))
data[:3]

[(13,
  "In CATAN (formerly The Settlers of Catan), players try to be the dominant force on the island of Catan by building settlements, cities and roads. On each turn dice are rolled to determine which resources the island produces. Players build structures by 'spending' resources (sheep, wheat, wood, brick and ore) which are represented by the relevant resource cards; each land type, with the exception of the unproductive desert, produces a specific resource: hills produce brick, forests produce wood, mountains produce ore, fields produce wheat, and pastures produce sheep.&#10;&#10;Set-up includes randomly placing large hexagonal tiles (each depicting one of the five resource-producing terrain types--or the desert) in a honeycomb shape and surrounding them with water tiles, some of which contain ports of exchange. A number disk, the value of which will correspond to the roll of two 6-sided dice, are placed on each terrain tile. Each player is given two settlements (think: houses) and

#### Shingling

For each game, convert its description into a set of hashed shingles.

1. Preprocessing
2. Shingles (n-grams)
3. Hashing


In [10]:
import re
import xxhash
import string

STOPWORDS = {
    "the","a","an","and","of","to","in","for","on","with","by","at","from",
    "this","that","these","those","players","player","play","game","games"
}

TRANS = {ord(c): ' ' for c in string.punctuation if c != '-'}
TRANS.update({ord(c): ' ' for c in string.ascii_uppercase})

def preprocess(text):
    if text is None:
        text = ""
    doc = nlp(text)
    return [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]


#def preprocess(text):
#    if not text:
#        return []
#    text = text.lower().translate(TRANS)
#    tokens = text.split()
#    return [t for t in tokens if t not in STOPWORDS]

def shingle_hash(s):
    return xxhash.xxh64(s).intdigest()

def shingles_for_one(gid, desc, n=3):
    tokens = preprocess(desc)
    out = set()
    limit = len(tokens) - n + 1

    for i in range(limit):
        s = tokens[i] + " " + tokens[i+1] + " " + tokens[i+2]
        out.add(shingle_hash(s))

    return out

def shingle_all(df, n=3):
    shingle_dict = {}
    ids = df["id"]
    descs = df["description"]

    for gid, desc in zip(ids, descs):
        shingle_dict[gid] = shingles_for_one(gid, desc, n)

    return shingle_dict


In [12]:
shingle_dict = shingle_all(df, n=3)


In [13]:

print("Number of shingles per game:")
for i, (g, s) in enumerate(shingle_dict.items()):
    if i > 10:
        break
    print(g, "->", len(s), "shingles")

Number of shingles per game:
13 -> 180 shingles
822 -> 88 shingles
30549 -> 114 shingles
68448 -> 123 shingles
167791 -> 194 shingles
266192 -> 74 shingles
173346 -> 134 shingles
230802 -> 79 shingles
178900 -> 86 shingles
36218 -> 113 shingles
9209 -> 117 shingles


### MinHash

Compress the large shingle set for each game into a small signature vector.

- Choose `k` hash functions:  
  `h_i(x) = (a_i * x + b_i) mod P`, with random `a_i`, `b_i`, and a large prime `P`.

- For each game and each hash function:
  - Apply hash function to each shingle in the game's set.
  - Take the minimum hashed value. This is the MinHash component for that hash function.


In [14]:
import numpy as np
import random

def build_minhash_functions(k, P=(1 << 61) - 1, seed=42):
    random.seed(seed)
    A = np.array([random.randint(1, P-1) for _ in range(k)], dtype=np.uint64)
    B = np.array([random.randint(0, P-1) for _ in range(k)], dtype=np.uint64)
    return A, B, np.uint64(P)

def minhash_signature(shingles, A, B, P):
    if not shingles:
        return np.full(len(A), P, dtype=np.uint64)

    x = np.fromiter(shingles, dtype=np.uint64)
    vals = (A[None, :] * x[:, None] + B[None, :]) % P
    return vals.min(axis=0)


In [15]:
def build_all_signatures(shingle_dict, k):
    A, B, P = build_minhash_functions(k)
    sigs = {}
    for gid, shs in shingle_dict.items():
        sigs[gid] = minhash_signature(shs, A, B, P)
    return sigs

In [16]:
sigs = build_all_signatures(shingle_dict, k=128)

In [17]:
print("signatures:", len(sigs), "length:", len(next(iter(sigs.values()))))

signatures: 27780 length: 128


#### Locality-Sensitive Hashing with banding

We use the classic MMDS banding technique:

- Signature length: `k`
- Choose number of bands `B` and rows per band `R` so that `B * R = k`
- For each game and each band:
  - Extract the band (a slice of the signature vector).
  - Hash the band to a bucket id.
  - Games whose bands hash to the same bucket are "candidates" to be similar.


In [18]:
def lsh_candidates(sigs, B=16):
    game_ids = list(sigs.keys())
    N = len(game_ids)
    k = len(next(iter(sigs.values())))
    R = k // B
    assert B * R == k, "k must be divisible by B"

    buckets = {}
    for gid, sig in sigs.items():
        for b in range(B):
            start = b * R
            band = tuple(sig[start:start+R])
            h = hash((b, band)) 
            if h not in buckets:
                buckets[h] = []
            buckets[h].append(gid)

    #Generate pairs
    pairs = set()
    for bucket_id, ids in buckets.items():
        if len(ids) < 2:
            continue
        ids = sorted(ids)
        for i in range(len(ids)):
            for j in range(i+1, len(ids)):
                pairs.add((ids[i], ids[j]))

    return pairs


In [19]:
lsh = lsh_candidates(sigs)
id_to_title = dict(zip(df["id"], df["name"]))


pairs_named = list(map(
    lambda ab: f"{id_to_title[ab[0]]} <-> {id_to_title[ab[1]]}",
    lsh
))

pairs_named[:10]


['Excalibohn <-> Bohnanza: 25th Anniversary Edition',
 'Anno Domini: Frauen <-> Anno Domini: Sex & Crime',
 'Exceed: Red Horizon – Reese, Heidi, Nehtali, and Vincent <-> Exceed: Red Horizon – Satoshi, Mei Lien, Morathi, and Baelkhor',
 "Funkoverse Strategy Game: Tim Burton's The Nightmare Before Christmas 100 <-> Funkoverse Strategy Game: Harry Potter 102",
 'Animal Upon Animal: Dinos <-> Animal Upon Animal: Unicorns',
 'Sherlock Far West: Disparos al amanecer <-> Sherlock: La copia',
 'Marvel United <-> Marvel United: Multiverse',
 'Q.E. <-> QE',
 'Anno Domini: Kirche & Staat <-> Anno Domini: Showbizz',
 "Rory's Story Cubes: Medic <-> Rory's Story Cubes: Animalia"]

In [20]:
def jaccard(A, B):
    inter = len(A & B)
    union = len(A | B)
    return inter / union if union > 0 else 0.0


def exact_jaccard_for_candidates(shingle_dict, candidates):
    out = []
    for a, b in candidates:
        s = jaccard(shingle_dict[a], shingle_dict[b])
        out.append((a, b, s))
    return out


In [21]:
jac_pairs = exact_jaccard_for_candidates(shingle_dict, lsh)
jac_pairs

used_ids = set()

for a, b in lsh:
    used_ids.add(a)
    used_ids.add(b)

print("Number of games in candidate graph:", len(used_ids))

Number of games in candidate graph: 575


In [ ]:
game_ids = sorted(list(used_ids))
id_to_index = {gid: i for i, gid in enumerate(game_ids)}
N = len(game_ids)

S = np.zeros((N, N), dtype=np.float32)

for a, b in lsh:
    i = id_to_index[a]
    j = id_to_index[b]
    sim = jaccard(shingle_dict[a], shingle_dict[b])
    S[i, j] = sim
    S[j, i] = sim

np.fill_diagonal(S, 0.0)   # INFO -> TECHNICALLY SHOULD BE 1, but does not work for my tool (gxpvis)

game_titles = [id_to_title[g] for g in game_ids]

with h5py.File(GEN_DATA / "gam_sim_small_lemma.h5", "w") as f:
    f.create_dataset("matrix", data=S.astype(np.float32))
    f.create_dataset(
        "node_names",
        data=np.array(game_titles, dtype=h5py.string_dtype())
    )

## full jaccard

note: just for fun, in general computationally infeasable to do, (28k games are not that much but this scales exponential)

In [20]:
import numpy as np

def build_full_jaccard_matrix(shingle_dict):

    keys = list(shingle_dict.keys())
    N = len(keys)

    sets = [shingle_dict[k] for k in keys]

    S = np.zeros((N, N), dtype=np.float32)

    for i in range(N):
        Ai = sets[i]
        for j in range(i + 1, N):
            Bj = sets[j]
            inter = len(Ai & Bj)
            union = len(Ai | Bj)
            sim = inter / union if union > 0 else 0.0
            S[i, j] = sim
            S[j, i] = sim

    return keys, S


jac_keys, jac_matrix = build_full_jaccard_matrix(shingle_dict)

out_file = GEN_DATA / "game_similarity_jaccard_full.h5"

with h5py.File(out_file, "w") as f:
    f.create_dataset("matrix", data=jac_matrix)
    f.create_dataset(
        "node_names",
        data=np.array([id_to_title[g] for g in jac_keys],
                      dtype=h5py.string_dtype())
    )

print("Wrote full Jaccard matrix to:", out_file)
print("Shape:", jac_matrix.shape)

Wrote full Jaccard matrix to: data\gen\game_similarity_jaccard_full.h5
Shape: (27780, 27780)


## full minhash Matrix

In [ ]:
import numpy as np

def full_minhash_matrix(sigs):
    keys = list(sigs.keys())
    N = len(keys)
    k = len(next(iter(sigs.values())))

    M = np.vstack([sigs[g] for g in keys])
    M = M.astype(np.uint64)
    print(M[:5])
    print(np.unique(M, axis=0).shape)
    S = np.zeros((N, N), dtype=np.float16)

    for i in range(N):
        eq = (M[i] == M[i+1:])
        sims = eq.mean(axis=1)
        S[i, i+1:] = sims
        S[i+1:, i] = sims

    np.fill_diagonal(S, 0) # technically it should be 1 but my tool is not working this way

    return keys, S


In [ ]:
keys, SimilarityMatrix = full_minhash_matrix(sigs)
print("full minhash matrix generated")

[[ 3510647397620425 15338983258644953  7121995958837106  9483587277079722
    113977077123849  2190669154861003 11626462023664658 13105635881063912
   4634925473994094  1795389916085611 11525273361811017  8149096543236479
   9297580216126661   628589923125487 17570903418896675   346469655300010
  30400694812124856  9395723247218144 14223009524313266  8138787345863659
    394034068379750  1587556536437164   126852099975916 10833595465114953
   2454622990788730 14039434445375588 17414770053555338   976112978663943
   6307150951703703 11406981646731525 19663037786490903   579512014608430
  12449599587530736   327818502420497  7247187925052831  7254894975403354
  22870273132020692  1313551037341979 26359733149581328  2691764095437160
   1118969214147179 15249378676166682  1296968529179661 10287122855389214
   5025901818632699 12753923639347110 13360574926194444 13758426534107462
  27659635138229980 10193612615026775  5942926913065043 12040114483020204
   5308562782580048 23379604592965645 

In [ ]:
print("SimilarityMatrix min/max:", SimilarityMatrix.min(), SimilarityMatrix.max())


S min/max: 0.0 1.0


### Storing as my format 
for use in gxpvis.skumantz.dev

In [45]:
id_to_title = dict(zip(df["id"], df["name"]))
game_titles = [id_to_title[g] for g in keys]


In [ ]:

with h5py.File(GEN_DATA / "game_similarity.h5", "w") as f:
    f.create_dataset("matrix", data=SimilarityMatrix)
    f.create_dataset("node_names", data=np.array(game_titles, dtype=h5py.string_dtype()))



### NLP using Transformer embeddings
we can make use of a sentence transformer to generate semantic embeddings

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer

descriptions = df["description"].fill_null("").to_list()

model = SentenceTransformer("all-MiniLM-L6-v2")


emb = model.encode( 
    descriptions,
    convert_to_numpy=True,
    normalize_embeddings=True,
    show_progress_bar=True
) # does all of the heavy lifting for us


sim_cosine = emb @ emb.T  # cosine similarity matrix via dot product between normalized embeddings

#sim_cosine = sim_cosine.astype(np.float32)
game_titles = df["name"].to_list() #extract the titles

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Batches: 100%|██████████| 869/869 [08:57<00:00,  1.62it/s]


In [ ]:
max_sim = sim_cosine.max(axis=1)

mean_sim = (sim_cosine.sum(axis=1) - 1.0) / (sim_cosine.shape[1]-1) # mean similarity to all other items excluding self-similarity (1)
threshold = 0.3 #/psi edge imilarity threshold used to count how many neighbors ar
degree = (sim_cosine > threshold).sum(axis=1) #number of items whose similarity exceds the threshold

score = 0.5 * max_sim + 0.3 * mean_sim + 0.2 * (degree / degree.max()) # score function weighing max similarity, mean similarity, and normalized degree# 

N = 1000 
idx = np.argsort(score)[-N:] #presevere only N best entries (to reduce size impact)


sim_small = sim_cosine[idx][:, idx] 
titles_small = np.array(game_titles)[idx]


In [78]:

with h5py.File(GEN_DATA / "game_similarity_transformer_small.h5", "w") as f:
    f.create_dataset("matrix", data=sim_small)
    f.create_dataset(
        "node_names",
        data=np.array(titles_small, dtype=h5py.string_dtype())
    )